# <font color=red>밑바닥부터 시작하는 딥러닝 1 - 한빛미디어, 사이토 고키</font>

### Contents (Chapter)

1. 헬로 파이썬 - 파이썬 기초 문법 소개, numpy, matplotlib
2. 퍼셉트론 - AND, NAND, OR 게이트
3. 신경망 - 활성화 함수, 다차원 배열 계산, 출력층 설계, MNIST
4. **`신경망 학습 - 손실 함수, 경사 하강법`**
5. 오차역전파법 - 역전파, 활성화 함수 구현
6. 학습 관련 기술들 - 매개변수 갱신, 배치 정규화, 하이퍼파라미터 값 찾기
7. 합성곱 신경망 (CNN) - 합성곱 계층, 풀링 계층, CNN 구현
8. 딥러닝 - 초기 역사, 딥러닝 활용
9. Appendix - Softmax with loss 계층의 계산 그래프

### 참고 자료

- [책 소개 링크](http://www.yes24.com/Product/Goods/34970929)
- [책 깃헙 링크](https://github.com/WegraLee/deep-learning-from-scratch)

---

## <font color=orange>Chapter 4. 신경망학습</font>

### 4.1 데이터에서 학습한다!

- 신경망의 특징은 데이터를 보고, 학습 할 수 있다는 점
- **학습**이란, 훈련 데이터로부터 가중치 매개변수의 최적값을 자동으로 획득하는 것
- 학습의 목표는 **손실 함수의 결과값을 가장 작게 만드는 가중치 매개변수를 찾는 것**

#### 4.1.1 데이터 주도 학습

- **기계학습**은 데이터에서 답을 찾고, 데이터에서 패턴을 발견하고 데이터로 이야기를 만드는 것
- 따라서 기계학습의 중심에는 **데이터**가 존재한다고 할 수 있음
- 기계학습에서는 `사람의 개입을 최소화`하면서 수집한 데이터로 패턴을 찾으려 시도
- MNIST 모델에서 숫자를 인식하는 알고리즘이 동작하기 위해서,
    - 이미지에서 특징 (Feature)을 추출하여 그 특징의 패턴을 기계학습 기술로 학습
    - **특징**은 입력 데이터에서 본질적인 데이터를 정확히 추출할 수 있도록 설계된 변환기
    - 이미지의 특징은 일반적으로 '벡터'로 변환하여 학습
- 기계학습에서는 모아진 데이터로부터 규칙을 찾아내는 역할을 **기계**가 한다고 할 수 있음
- 사람이 직접 설계하는 것에 비해서 부담은 적지만, 이미지를 벡터로 변환하는 등의 과정은 사람이 해야 함
- 문제에 적합한 특징을 설계하지 (전처리) 못한다면, 좋은 결과를 획득하기도 어렵다고 할 수 있음
- 딥러닝을 입력부터 출력까지 사람의 개입 없이 동작하여 **종단간 기계학습 (end-to-end machine learning)**이라고도 함

#### 4.1.2 훈련 데이터와 시험 데이터

- 기계학습 문제는 **훈련 데이터 (Training data)와 시험 데이터 (Test data)**로 나눠 학습과 실험을 수행
- 훈련 데이터를 통해서 학습하면서 최적의 매개변수를 찾음
- 그 후에 시험 데이터를 사용하여 앞서 훈련한 모델의 성과를 평가
- 결국 모델링의 궁극적인 목적은 범용적으로 사용할 수 있는 **일반화된 모델**이므로, 훈련 데이터와 시험 데이터로 구분
- 이를 위해서 한번도 보지 못했던 훈련에 포함되지 않은 데이터로 성능을 측정
- 한 데이터셋에만 지나치게 최적화 된 상태를 **오버피팅 (Overfitting)**이라고 함

### 4.2 손실 함수

- 신경망 학습에서는 현재의 상태를 `하나의 지표`로 표현
- 그 지표를 가장 좋게 만들어주는 가중치 매개변수 값을 찾는 것이 목적
- 신경망 학습에서 사용하는 지표는 **손실 함수 (Loss function)**이라고 하는데, 일반적으로 `오차제곱합과 교차 엔트로피 오차`를 사용
    - 손실 함수는 신경망의 성능의 나쁨을 나타내는 지표
    - 즉, 현재 신경망이 훈련 데이터를 얼마나 잘 처리하지 못하는지의 성능을 담고 있음
    - 따라서 손실 함수의 값이 클수록 좋지 않는 성능을 갖고 있다고 할 수 있음
    
#### 4.2.1 오차제곱합 (Sum of Square for Error, SSE)

<br/>

$$
E = \frac{1}{2}{\sum_{k} (y_k - t_k)^2}
$$

<br/>

- 가장 많이 쓰이는 손실 함수는 **오차제곱합**이고, 수식은 위와 같음
- $y_k$는 신경망의 출력 (신경망이 출력한 값), $t_k$는 정답 레이블, $k$는 데이터의 차원 수를 나타냄
- 즉, 오차제곱합은 각 원소의 출력과 정답 레이블의 차를 제곱한 후에 그 총합을 구하는 것
- 파이썬 코드로 오차제곱합을 구현하면, 아래와 같음을 알 수 있음
    - MNIST에서 정답이 '2'라고 했을 때, 모델의 출력 결과가 맞았을 때와 틀렸을 때의 손실함수 값 확인
    - 출력 결과가 맞을 때, 손실 함수 값이 0.0975로 작게 나오는 것을 확인할 수 있음
    - 반면, 출력 결과가 틀렸을 때, 손실 함수 값이 0.5975로 크게 나오는 것을 확인할 수 있음
    - 즉, `손실 함수의 값이 작을수록 정답에 수렴한다`고 할 수 있음

In [1]:
import numpy as np

def sum_squares_error(y, t):
    return 0.5 * np.sum((y - t)**2)

# 정답은 '2'
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

# 예1: '2'일 확률이 가장 높을 때
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
sum_squares_error(np.array(y), np.array(t)) # 0.09750000000000003

# 예2: '7'일 확률이 가장 높을 때
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
sum_squares_error(np.array(y), np.array(t)) # 0.5975

0.5975

#### 4.2.2 교차 엔트로피 오차 (Cross Entropy Error, CEE)

<br/>

$$
E = -{\sum_{k} t_klog_ey_k}
$$

<br/>

- 위 수식은 교차 엔트로피 오차의 수식
- log는 밑이 e인 자연로그, $y_k$는 신경망의 출력, $t_k$는 정답 레이블이면서 정답 인덱스만 1 (원-핫 인코딩)
- 따라서 실질적으로 정답일 때 추정의 자연로그를 계산하는 식 (다른 경우에는 모두 0이기 때문)
- 즉, `교차 엔트로피 오차는 정답일 때의 출력이 전체 값을 결정`
- 파이썬 코드로 교차 엔트로피 오차를 구현하면, 아래와 같음을 알 수 있음
    - MNIST에서 정답이 '2'라고 했을 때, 모델의 출력 결과가 맞았을 때와 틀렸을 때의 손실함수 값 확인
    - 출력 결과가 맞을 때, 손실 함수 값이 0.5108로 작게 나오는 것을 확인할 수 있음
    - 반면, 출력 결과가 틀렸을 때, 손실 함수 값이 2.3025로 크게 나오는 것을 확인할 수 있음
    - 즉, 위의 오차제곱합과 동일하게 `손실 함수의 값이 작을수록 정답에 수렴한다`고 할 수 있음

In [2]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

# 정답은 '2'
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

# 예1: '2'일 확률이 가장 높을 때
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t)) # 0.510825457099338

# # 예2: '7'일 확률이 가장 높을 때
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t)) # 2.3025840929945454

2.3025840929945454

#### 4.2.3 미니배치 학습

<br/>

$$
E = -\frac{1}{N}{\sum_{n}}{\sum_{k} t_klog_ey_k}
$$

<br/>

- 위 수식은 **훈련 데이터 모두**에 대한 손실 함수 값을 구하는 수식
- 앞선 수식과 비슷하고, 데이터 하나에 대한 손실 함수를 단순히 N개의 데이터로 확장한 것
- 마지막에는 N으로 나눠서 정규화 (평균 손실 함수의 역할)
- 하지만 현실적으로 **빅데이터 안에서 이 모든 데이터를 대상으로 값을 계산하는 것은 비효율적**
- 따라서 데이터 일부를 추려 전체의 근사치로 이용하는 **미니배치 (Mini-batch)** 방법을 사용 (일부만 골라 학습)
- 아래 코드는 MNIST 데이터셋에서 np.random.choice 함수를 통해 미니배치로 계산하는 과정을 소개

In [3]:
import sys, os, pickle
github_url = '/Users/paul/Desktop/github/deep-learning-from-scratch-master/'
sys.path.append(github_url)
import numpy as np
from dataset.mnist import load_mnist

### MNIST 데이터셋 불러오기
(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=True)

x_train.shape # (60000, 784)
t_train.shape # (60000, 10)

### 훈련 데이터에서 무작위로 10장만 추출하기
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

#### 4.2.4 (배치용) 교차 엔트로피 오차 구현하기